In [7]:
import torch
import torchvision.models as models
from torchvision import transforms
from PIL import Image

# ResNet-50 모델을 불러옵니다. 'pretrained=True'를 사용하여 ImageNet에서 사전 학습된 가중치를 불러옵니다.
model = models.resnet50(pretrained=True)


In [8]:
# 평가 모드로 전환
model.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
# 3. 더미 데이터 생성 (큰 사이즈 이미지 생성)
# 큰 이미지 크기 (예: 1024x1024)
large_image_size = (1024, 1024)

# 더미 데이터를 만들기 위해 임의의 값으로 채워진 큰 이미지를 생성합니다.
# PIL 이미지를 생성 후 나중에 텐서로 변환합니다.
large_dummy_image = Image.fromarray((torch.randn(*large_image_size, 3) * 255).numpy().astype('uint8'))

In [10]:
# 4. 이미지 전처리
# 전처리 파이프라인 정의: 이미지 크기 조정, 중앙 크롭, 텐서 변환, 정규화
preprocess = transforms.Compose([
    transforms.Resize(256),                # 256x256으로 크기 조정
    transforms.CenterCrop(224),            # 224x224로 중앙 크롭
    transforms.ToTensor(),                 # 텐서로 변환
    transforms.Normalize(                  # 정규화 (ImageNet 평균 및 표준편차 사용)
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
])
input_tensor = preprocess(large_dummy_image)
input_batch = input_tensor.unsqueeze(0)


In [11]:
# 5. 모델에 더미 데이터 입력하고 예측 수행
with torch.no_grad():  # 그래디언트 계산 비활성화
    output = model(input_batch)

In [12]:
# 6. 출력 확인하기
print("Output shape:", output.shape)  # 출력 벡터의 크기
print("Output (first 5 values):", output[0][:5])  # 출력 벡터의 상위 5개 값


Output shape: torch.Size([1, 1000])
Output (first 5 values): tensor([-1.9111, -0.7645,  0.3483,  0.6148,  0.4052])
